In [48]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, TensorDataset
import pickle
import torch.nn.functional as F
import pandas as pd

In [36]:
# Load the dictionaries
with open('models/bilstm_char/id2word.pkl', 'rb') as f:
    id2word = pickle.load(f)
with open('models/bilstm_char/id2tag.pkl', 'rb') as f:
    id2tag = pickle.load(f)
with open('models/bilstm_char/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)
with open('models/bilstm_char/nertags.pkl', 'rb') as f:
    nertags = pickle.load(f)
with open('models/bilstm_char/char_vocab.pkl', 'rb') as f:
    char_vocab = pickle.load(f)

# Load pretrained embeddings
word_embeds = torch.load('models/bilstm_char/pretrained_embeddings.pt')
char_onehot = torch.load('models/bilstm_char/pretrained_char_embeddings.pt')

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [38]:
## LSTM for CHARACTER level

class forLSTM(nn.Module):
    def __init__(self, embedding_size, hidden_size, pretr_char_embed):
        super(forLSTM, self).__init__()
        self.charembed = nn.Embedding.from_pretrained(pretr_char_embed, freeze = False) #size of pretrained = (totalchars,embedding size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, bidirectional = True, batch_first = True)

    def forward(self, xchar, xlength_char):
        shape = xchar.shape
        xchar = xchar.view(-1, shape[2])
        xlength_char = xlength_char.view(-1)
        
        input = pack_padded_sequence(xchar, xlength_char.cpu(), batch_first=True, enforce_sorted=False)
        input, _ = pad_packed_sequence(input, batch_first=True)
        embed = self.charembed(input)
        _, (h,_) = self.lstm(embed) #h is of size (2, 128*maxno. of words in a sentence in the batch, 25)
        h = h.view(h.shape[1], 50)
        h = h.view(shape[0], shape[1], 50)
        return h

In [39]:
## BILSTM model

class BiLSTM(nn.Module):
    def __init__(self, embedding_size, hidden_size, total_words, num_class, pretrained = False, pretrained_embed = None, char_embed_size = 0, pretr_char_embed = None):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.wordembed = nn.Embedding.from_pretrained(pretrained_embed, freeze = False)
        self.for_charembed = forLSTM(embedding_size = char_embed_size, hidden_size = 25, pretr_char_embed = pretr_char_embed)
        self.dropout = nn.Dropout(p = 0.5)
        self.bilstm = nn.LSTM(embedding_size + 50,hidden_size, bidirectional = True, batch_first = True)
        self.linear = nn.Linear(2*hidden_size, num_class) # 2 because forward and backward concatenate

    def forward(self, x, xchar, xlengths, xlength_char):
        x = pack_padded_sequence(x, xlengths.cpu(), batch_first=True, enforce_sorted=False)
        x, _ = pad_packed_sequence(x, batch_first=True)

        xlength_char = pack_padded_sequence(xlength_char, xlengths.cpu(), batch_first=True, enforce_sorted=False)
        xlength_char, _ = pad_packed_sequence(xlength_char, batch_first=True, padding_value = len("<pad>")) 
        # above this line padding value is taken as len of pad word becasue that is what we pad sentences 
        # with hance at a character level it should be the length

        xchar = pack_padded_sequence(xchar, xlengths.cpu(), batch_first=True, enforce_sorted=False)
        xchar, _ = pad_packed_sequence(xchar, batch_first=True)

        word_embedding = self.wordembed(x) # x is of size(batchsize, seq_len), wordembed is of size (batchsize, seq_len, embedding_size = 100)
        forwardchar= self.for_charembed(xchar, xlength_char) #forwardchar and backwardchar would be of size (batchsize, seqlen. embedding_size = 25each) 
        word_embedding = torch.cat((word_embedding, forwardchar), dim = 2)

        word_embedding = self.dropout(word_embedding) #dropout
        out, (h,c) = self.bilstm(word_embedding) #'out' has dimension(batchsize, seq_len, 2*hidden_size)

        out = self.linear(out) #now 'out' has dimension(batchsize, seq_len, num_class)
        out = out.view(-1, out.shape[2]) # shape (128*seqlen, 18)
        out = F.log_softmax(out, dim=1) # take the softmax across the dimension num_class, 'out' has dimension(batchsize, seq_len, num_class)
        return out

In [40]:
def load_char_data(words, charvocab):
    train_char_sent = []
    train_char_label = []
    for word in words:
        chars = []
        char_labels = []

        word_sep = list(word)
        for c in word_sep[:-1]:
            if (c in charvocab.keys()):
                chars.append(charvocab[c])
            else:
                chars.append(charvocab['<oovchar>'])
        for c in word_sep[1:]:
            if (c in charvocab.keys()):
                char_labels.append(charvocab[c])
            else:
                char_labels.append(charvocab['<oovchar>'])
        
        train_char_sent.append(chars)
        train_char_label.append(char_labels)

    # padding the char_sents at the end
    seq_maxlen = max(len(x) for x in train_char_sent)
    x_lengths_char = [len(x) for x in train_char_sent]
    Xtrain_char = []
    Ytrain_char = []
    for char_sent, char_label in zip(train_char_sent, train_char_label):
        length_toappend = seq_maxlen - len(char_sent)
        Xtrain_char.append(char_sent+[0]*length_toappend)
        Ytrain_char.append(char_label+[0]*length_toappend) # 0 is padchar


    Xtrain_char = torch.Tensor(Xtrain_char)
    Ytrain_char = torch.Tensor(Ytrain_char)
    x_lengths_char = torch.Tensor(x_lengths_char)
    # print(Xtrain.shape, Ytrain.shape, x_lengths.shape)
    
    return Xtrain_char, Ytrain_char, x_lengths_char

def pad_chars(topadlist, maxlen):
    topadlist = topadlist + [0]*(maxlen-len(topadlist))

    return topadlist

def make_id2word_charvocab(vocab, charvocab):
    max_charlen = max(len(word) for word in vocab.keys())
    word_charlevel_vocab = {}
    wordid2wordlen = {}
    for word in vocab.keys():
        word_charlevel_vocab[vocab[word]] = [charvocab[w] if w in charvocab.keys() else charvocab['<oovchar>'] for w in word]
        word_charlevel_vocab[vocab[word]] = pad_chars(word_charlevel_vocab[vocab[word]], max_charlen)

        wordid2wordlen[vocab[word]] = len(word)
        # word_charlevel_vocab[vocab[word]] = word_charlevel_vocab[vocab[word]].extend([charvocab['<padchar>']]*(max_charlen-len(word_charlevel_vocab[vocab[word]])))
    return word_charlevel_vocab, wordid2wordlen


def load_char_level(X, wordid2word_charlevel_vocab, wordid2wordlen):
    #X is of shape (no.of.sentences, 104)
    Xcharlevel = [] # will finally be fo shape (total.sentences, max_sent.len, )
    Xcharlevel_lengths = []
    for i in range(X.shape[0]):
        sentence = []
        wordlengths = []
        for j in range(X.shape[1]):
            sentence.append(torch.tensor([wordid2word_charlevel_vocab[int(X[i, j].item())]]))
            wordlengths.append(wordid2wordlen[int(X[i, j].item())])
            # sentences = pad_sequence(sentences)
        # print(i)
        Xcharlevel_lengths.append(wordlengths)
        Xcharlevel.append(torch.stack(sentence))
    
    return torch.squeeze(torch.stack(Xcharlevel)), torch.tensor(Xcharlevel_lengths)

def get_charvocab(vocab):
    # using vocab make charvocab
    words = list(vocab.keys())
    characters = [char for word in words for char in word]
    characters = list(set(characters))
    char_vocab = {}
    char_vocab["<padchar>"] = 0
    char_vocab["<oovchar>"] = 1
    for i, char in enumerate(characters):
        char_vocab[char] = i+2

    return char_vocab


In [41]:
def test_load_data(datapath, buildvocab_tags= True, vocab = None, nertags = None):
    if(buildvocab_tags == True):
        all_words = []
        all_tags = []
        with open(datapath) as f:
            lines = f.readlines()
            sent_num = 0
            for line in lines: #1: so that the first blank line isn't taken into account
                if(line == "\n"):
                    sent_num+=1
                else:
                    line_sep = line.split('\t')
                    all_words.append(line_sep[0])
                    all_tags.append(line_sep[1][:-1])
                    
        words = list(set(all_words))
        tags = list(set(all_tags))

        vocab = {}
        vocab['<pad>'] = 0 # for padding input sequences
        vocab['<oov>'] = 1
        for i, word in enumerate(words):
            vocab[word] = i+2
            
        nertags = {}
        nertags['padtag'] = 0
        for i,nertag in enumerate(tags):
            nertags[nertag] = i+1

    train_sent = []
    train_tags = []
    with open(datapath) as f:
        lines = f.readlines()
        sent_num = 0
        sentence = []
        tag = []
        for line in lines: #1: so that the first blank line isn't taken into account
            if(line == "\n"):
                sent_num+=1
                train_sent.append(sentence)
                train_tags.append(tag)
                sentence = []
                tag = []
            else:
                line_sep = line.split("\t")
                if(line_sep[0] in vocab.keys()):
                    sentence.append(vocab[line_sep[0]])
                else:
                    sentence.append(vocab['<oov>'])  
                tag.append(nertags[line_sep[-1][:-1]])

    # padding the sentences at the end
    seq_maxlen = max(len(x) for x in train_sent)
    x_lengths = [len(x) for x in train_sent]
    Xtrain = []
    Ytrain = []
    for sent, tags in zip(train_sent, train_tags):
        length_toappend = seq_maxlen - len(sent)
        Xtrain.append(sent+[0]*length_toappend)
        Ytrain.append(tags+[0]*length_toappend)


    Xtrain = torch.Tensor(Xtrain)
    Ytrain = torch.Tensor(Ytrain)
    x_lengths = torch.Tensor(x_lengths)
    
    return Xtrain, Ytrain, x_lengths, vocab, nertags


In [42]:
model = BiLSTM(embedding_size = 100, hidden_size = 100, total_words = len(vocab), num_class = len(nertags),  pretrained=True, pretrained_embed = word_embeds, char_embed_size = len(char_vocab),pretr_char_embed = char_onehot) 
model.load_state_dict(torch.load('models/bilstm_char/trained_bilstm_model_state_dict.pth'))

model.eval()

BiLSTM(
  (wordembed): Embedding(23626, 100)
  (for_charembed): forLSTM(
    (charembed): Embedding(88, 88)
    (lstm): LSTM(88, 25, batch_first=True, bidirectional=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (bilstm): LSTM(150, 100, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=200, out_features=10, bias=True)
)

In [43]:
def loader(testdatapath):
    Xtest, Ytest, x_testlengths, _, _ = test_load_data(testdatapath, buildvocab_tags=False, vocab = vocab, nertags = nertags)

    Xtest_temp, Ytest_temp, x_testlengths_temp, testvocab, testnertags = test_load_data(testdatapath, buildvocab_tags=True)
    wordid2word_charlevel_vocab_test, wordid2wordlen_test = make_id2word_charvocab(testvocab, char_vocab) # of the form {word:[1,2,3,4]}, {wordnum:wordlen}
    #make char level train data for the char embeddings 
    Xtest_char, xtestlength_char = load_char_level(Xtest_temp, wordid2word_charlevel_vocab_test, wordid2wordlen_test)
    #finally make the dataloader for train
    testdataset = TensorDataset(Xtest, Xtest_char, Ytest, x_testlengths, xtestlength_char)
    loader_test = DataLoader(testdataset, batch_size= 1, shuffle=False)
    return loader_test

In [44]:
def test_model(model, loader):
    y_predicted = []
    with torch.no_grad():
        for step, (X, Xchar, Y, xlen, xlen_char) in enumerate(loader):
            ypred = model(X.long().to(device), Xchar.to(device), xlen.to(device), xlen_char.to(device))#.permute(0, 2, 1)
            ypred = torch.argmax(ypred.to('cpu'), dim = 1)
            ypred = ypred.view(Y.shape[0], -1)
            y_predicted.append(ypred)

    y_predicted_list = []
    for i in range(len(y_predicted)):
        for j in range(y_predicted[i].shape[0]):
            sent_pred = []
            for x in range(y_predicted[i].shape[1]):
                sent_pred.append(id2tag[int(y_predicted[i][j, x])])
            y_predicted_list.append(sent_pred)
    return y_predicted_list

In [45]:
def writefile(testfilepath, outputfilepath, predictions):
    final_output = [] #list of lists which will finally be written to file
    with open(testfilepath) as f:
        lines = f.readlines()
        sentnum = 0 #to take care of the first blank line
        wordnum = 0
        for line in lines:
            if(line == '\n'):
                sentnum+=1
                wordnum = 0
                final_output.append(line)

            else:
                line_sep = line.split(sep = "\t")
                word = line_sep[0]
                prediction = predictions[sentnum][wordnum]
                final_output.append(f'{word}\t{prediction}\n')
                wordnum+=1
    #write the outputfilepath
    with open(outputfilepath, 'w+') as f:
        f.writelines(final_output)

In [46]:
# SPAN-F1 score
def readBIO(path):
    ents = []
    curEnts = []
    for line in open(path):
        line = line.strip()
        if line == '':
            ents.append(curEnts)
            curEnts = []
        elif line[0] == '#' and len(line.split('\t')) == 1:
            continue
        else:
            curEnts.append(line.split('\t')[1])
    return ents

def toSpans(tags):
    spans = set()
    for beg in range(len(tags)):
        if tags[beg][0] == 'B':
            end = beg
            for end in range(beg+1, len(tags)):
                if tags[end][0] != 'I':
                    break
            spans.add(str(beg) + '-' + str(end) + ':' + tags[beg][2:])
            #print(end-beg)
    return spans

def getInstanceScores(predPath, goldPath):
    goldEnts = readBIO(goldPath)
    predEnts = readBIO(predPath)
    entScores = []
    tp = 0
    fp = 0
    fn = 0
    for goldEnt, predEnt in zip(goldEnts, predEnts):
        goldSpans = toSpans(goldEnt)
        predSpans = toSpans(predEnt)
        overlap = len(goldSpans.intersection(predSpans))
        tp += overlap
        fp += len(predSpans) - overlap
        fn += len(goldSpans) - overlap
        
    prec = 0.0 if tp+fp == 0 else tp/(tp+fp)
    rec = 0.0 if tp+fn == 0 else tp/(tp+fn)
    f1 = 0.0 if prec+rec == 0.0 else 2 * (prec * rec) / (prec + rec)
    return f1

In [47]:
noise_rates = {
    'capitalization_swap': [0.1, 0.15, 0.2, 0.25, 0.3], 
    'character_swap': [0.1, 0.15, 0.2, 0.25, 0.3],
    'character_removal': [0.1, 0.15, 0.2, 0.25, 0.3],
    'character_replacement': [0.1, 0.15, 0.2, 0.25, 0.3]
}

In [49]:
for noise_type in noise_rates.keys():
    for rate in noise_rates[noise_type]:
        path = f'data/altered_2/{noise_type}_rate_{rate}.txt'
        my_loader = loader(path)
        predictions = test_model(model, my_loader)
        #print(predictions)
        writefile(path, f'predictions/altered_2/char/{noise_type}_rate_{rate}.txt',predictions)

In [50]:
types = []
rates = []
f1_score = []

for noise_type in noise_rates.keys():
    for rate in noise_rates[noise_type]:
        score = getInstanceScores(f'predictions/altered_2/char/{noise_type}_rate_{rate}.txt', 'data/gold.txt')
        
        # Append data to lists
        types.append(noise_type)
        rates.append(rate)
        f1_score.append(score)

results = {'Type': types, 'Rate': rates, 'F1 Score': f1_score}
df = pd.DataFrame(results)
print(df)

df.to_csv('out/df_altered_2_char.csv', index=False)

                     Type  Rate  F1 Score
0     capitalization_swap  0.10  0.681053
1     capitalization_swap  0.15  0.654251
2     capitalization_swap  0.20  0.624754
3     capitalization_swap  0.25  0.591449
4     capitalization_swap  0.30  0.584604
5          character_swap  0.10  0.702227
6          character_swap  0.15  0.561937
7          character_swap  0.20  0.436957
8          character_swap  0.25  0.387277
9          character_swap  0.30  0.368905
10      character_removal  0.10  0.698443
11      character_removal  0.15  0.538231
12      character_removal  0.20  0.395190
13      character_removal  0.25  0.349141
14      character_removal  0.30  0.346803
15  character_replacement  0.10  0.701609
16  character_replacement  0.15  0.538052
17  character_replacement  0.20  0.393503
18  character_replacement  0.25  0.339022
19  character_replacement  0.30  0.340380


In [51]:
for noise_type in noise_rates.keys():
    for rate in noise_rates[noise_type]:
        path = f'data/altered_3/{noise_type}_rate_{rate}.txt'
        my_loader = loader(path)
        predictions = test_model(model, my_loader)
        #print(predictions)
        writefile(path, f'predictions/altered_3/char/{noise_type}_rate_{rate}.txt',predictions)

In [52]:
types = []
rates = []
f1_score = []

for noise_type in noise_rates.keys():
    for rate in noise_rates[noise_type]:
        score = getInstanceScores(f'predictions/altered_3/char/{noise_type}_rate_{rate}.txt', 'data/gold.txt')
        
        # Append data to lists
        types.append(noise_type)
        rates.append(rate)
        f1_score.append(score)

results = {'Type': types, 'Rate': rates, 'F1 Score': f1_score}
df = pd.DataFrame(results)
print(df)

df.to_csv('out/df_altered_3_char.csv', index=False)

                     Type  Rate  F1 Score
0     capitalization_swap  0.10  0.678354
1     capitalization_swap  0.15  0.657219
2     capitalization_swap  0.20  0.625711
3     capitalization_swap  0.25  0.594132
4     capitalization_swap  0.30  0.570690
5          character_swap  0.10  0.703132
6          character_swap  0.15  0.564339
7          character_swap  0.20  0.438082
8          character_swap  0.25  0.376210
9          character_swap  0.30  0.373736
10      character_removal  0.10  0.698628
11      character_removal  0.15  0.539425
12      character_removal  0.20  0.397400
13      character_removal  0.25  0.348741
14      character_removal  0.30  0.347168
15  character_replacement  0.10  0.700847
16  character_replacement  0.15  0.537385
17  character_replacement  0.20  0.390278
18  character_replacement  0.25  0.338835
19  character_replacement  0.30  0.338558


In [34]:
noise_type = 'capitalization_swap'
rate = 0.1

testdatapath = f'data/altered/{noise_type}_rate_{rate}.txt'
prediction_path = f'predictions/bilstm_char/{noise_type}_rate_{rate}.txt'

my_loader = loader(testdatapath)
predictions = test_model(model, my_loader)
writefile(testdatapath, prediction_path, predictions)

span_f1_score = getInstanceScores(prediction_path,'data/gold.txt')
print('Span-F1 score: ', span_f1_score)

Span-F1 score:  0.6810616284300495
